<a href="https://colab.research.google.com/github/bptripp/ai-course/blob/main/ehr_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To summarize what we have seen so far, a simple way to transform words into vectors is one-hot encoding, and a simple way to combine word vectors from a passage of text is to add them together. These simple methods have three important limitations. Let's deal with them one-by-one.

The code below loads a pre-trained BERT network that produces lists of word vectors from text.

In [ ]:
#!pip install boto3 sentencepiece sacremoses huggingface_hub tokenizers
import torch
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)

# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)

print(type(encoded_layers))
print(tokens_tensor)


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

<class 'str'>
tensor([[  101,  2627,  1108,  3104,  1124, 15703,   136,   102,  3104,  1124,
         15703,  1108,   170, 16797,  8284,   102]])


#Encoding an Electronic Health Record
It is an open question how best to encode a complex electronic health record, with multiple orders, lab results, notes, etc. but it is clearly important to do so. A machine learning model may need all of this information to make the best inferences. For example, in the recent literature, ICD discharge codes have been inferred more accurately by systems that use the full record of a hospital visit than by systems that use only a discharge letter as input.

One approach begins with creating an extended vocabulary that includes medications, tests, and results. A vector is then created for every item in the EHR, using a model like BERT. Each word in a note is encoded in this way, and orders and results are encoded as well. This may result in thousands of vectors for a patient visit. The vectors are then divided into groups according to the times they were entered in the EHR, for example in six-hour blocks. All the vectors in a group are summed (similar to a bag of words encoding). This results in a list of vectors that represents a timeline of the patient's stay. This list of vectors serves as input to deep networks that predict things like discharge codes, length of stay, and risk of mortality.

We have already seen how to encode notes as vectors, but let's also consider how we could encode a 12-lead ECG order. Suppose the EHR contains the following information about the order: time of order, ordering physician, reference event (e.g. exercise, medication), indications, patient posture, and comments. Most of these are selected from drop-down lists in the EHR interface at the time of order. Each item in each drop-down list can be added to our vocabulary and assigned a vector. Here we have some technical decisions to make. For example, should we include the ordering physician in the vocabulary? If so, how? Advanced embeddings such as Word2Vec and BERT require training on text passages, but we may not have suitable passages for this purpose. A simple way forward would be to use the vector for "doctor" but change it a little bit at random for each different doctor. Having made such decisions, we could encode the order as either a sum or list of the vectors for each field.

The code to do all of the above would be too complicated to review here, but let's look at a simplified example. We will read a patient's orders from an electronic health record, encode the description of each order, and add all the resulting vectors together to create a single vector that summarizes the orders.

Let's start by downloading a pretrained word embedding. This one is similar to Word2Vec, and should only take a few seconds to download. We'll print an example vector from this embedding, for the word "cortisol".

In [ ]:
import gensim.downloader
vectors = gensim.downloader.load('glove-twitter-25')
print(vectors.get_vector('cortisol'))


[-0.042121 -0.30836  -1.2449    2.0054    0.82295   2.0069   -0.20042
 -0.73573   1.2235    0.013556  0.47971   0.78728  -0.43291   0.77171
  0.46065   1.0751   -0.73819   1.2626    1.9331   -0.31433  -0.46394
  0.90599   0.67538  -0.44131  -0.86128 ]
25


The next step is to query a patient's orders from an electronic health record (EHR) system. To make this more realistic, we will use the HL7 FHIR protocol and query a fictional EHR. To do this, we need to import a package called "fhirclient". However, fhirclient is not yet installed on the server where you are running this code. The following line of code will install it.

In [ ]:
!pip install git+https://github.com/smart-on-fhir/client-py.git

  Cloning https://github.com/smart-on-fhir/client-py.git to /tmp/pip-req-build-elzhb7jh
  Running command git clone -q https://github.com/smart-on-fhir/client-py.git /tmp/pip-req-build-elzhb7jh
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 41 kB 634 kB/s 
  Created wheel for fhirclient: filename=fhirclient-4.0.0-py2.py3-none-any.whl size=683007 sha256=76d520d27433a6a0cf7114556fcf781a70f1931fbcd034b5d3f14c85fe989225
  Stored in directory: /tmp/pip-ephem-wheel-cache-meg5_9t8/wheels/0a/b6/d6/645668a711b21c1934ecc4acb01f1fc9799afe8da3c9ee8f44
Successfully built fhirclient


Now we can import fhirclient and get ready to query the EHR.

In [ ]:
import numpy as np
from fhirclient import client
import fhirclient.models.servicerequest as sr

settings = {
    'app_id': 'EHR',
    'api_base': 'http://hapi.fhir.org/baseR4/'
}
ehr = client.FHIRClient(settings=settings)


We will need a new "get_word_vector" function that uses the pretrained embedding that we downloaded.

In [ ]:
vector_length = 25 # our embedding vectors are 25 numbers long

def get_word_vector(word):
  if vectors.__contains__(word):
    return vectors.get_vector(word)
  else:
    return np.zeros(vector_length)


We are nearly done. We now use a patient's medical record number to retrieve their service requests (orders). Finally, we loop through the orders, find the vector embedding of every word in each order description, and add these together. This code may take a few seconds to run.

In [ ]:
import re

patient_mrn = '2782378'
bundle = sr.ServiceRequest.where(struct={'subject': patient_mrn}).perform(ehr.server)

order_vector = np.zeros(vector_length) # start with a vector of zeros
for order in bundle.entry:
    description = order.resource.code.coding[0].display
    description = re.sub('[^\w ]', '', description)
    words = description.lower().split()
    for word in words:
      order_vector = order_vector + get_word_vector(word)

print(order_vector)


[-3.41879508e+00  2.97042598e+00 -4.69604304e+00  2.71748985e+00
  9.12437401e+00 -3.16932946e-01  1.03945917e+01 -1.56265371e+01
  6.67294295e+00  5.22118399e+00  3.66568091e+00  4.83383909e+00
 -6.94769299e+01  4.13983300e+00  6.46364304e+00 -8.32750037e-01
  8.50220607e+00 -3.12261239e-01  4.84719276e-02 -3.53599308e+00
 -1.04554601e+01 -7.76211061e-01  2.37491991e+00 -4.34321847e-01
 -5.88022403e+00]


This list of numbers looks meaningless to us, but it contains a concise summary of the patient's orders in exactly the form that a deep network needs. This is not so strange. Consider that our brains receive information only in the form of patterns of action potentials from sensory organs. What we have created here is something like a sensory organ for a deep network. Not one that senses sights or sounds, but one that senses electronic health records directly.